In [38]:
import wfdb
import numpy as np
import pandas as pd

In [39]:
def heart_rate(annotation):
    r_peaks = annotation.sample
    rr_intervals = np.diff(r_peaks) / annotation.fs

    heart_rate = 60 /rr_intervals
    return r_peaks[1:], heart_rate

def respiration_rate(annotation):
    resp_peaks = annotation.sample
    resp_intervals = np.diff(resp_peaks) / annotation.fs

    respiration_rate = 60 / resp_intervals
    return resp_peaks[1:], respiration_rate


def process_infant_data(infant_num):
    ecg_record = wfdb.rdrecord(f'infant{infant_num}_ecg')
    sampling_rate_ecg = ecg_record.fs
    ecg_signal = ecg_record.p_signal

    resp_record = wfdb.rdrecord(f'infant{infant_num}_resp')
    sampling_rate_resp = resp_record.fs
    resp_signal = resp_record.p_signal

    r_peak_annotations = wfdb.rdann(f'infant{infant_num}_ecg', 'qrsc')
    r_times, heart_rates = heart_rate(r_peak_annotations)

    resp_peak_annotations = wfdb.rdann(f'infant{infant_num}_resp', 'resp')
    resp_times, respiration_rates = respiration_rate(resp_peak_annotations)

    # Truncate or pad respiration peaks to match heart rate length
    if len(resp_times) > len(heart_rates):
        resp_times = resp_times[:len(heart_rates)]
        respiration_rates = respiration_rates[:len(heart_rates)]
    else:
        avg_resp_peak = np.mean(resp_times) if len(resp_times) > 0 else 0
        resp_times = np.pad(resp_times, (0, len(heart_rates) - len(resp_times)), 'constant', constant_values=avg_resp_peak)
        avg_resp_rate = np.mean(respiration_rates) if len(respiration_rates) > 0 else 0
        respiration_rates = np.pad(respiration_rates, (0, len(heart_rates) - len(respiration_rates)), 'constant', constant_values=avg_resp_rate)

    data = pd.DataFrame({
        'time_r_peak(s)': r_times/ecg_record.fs,
        'heart_rate (BPM)': heart_rates,
        'respiration_peak_time(s)': resp_times/ resp_record.fs,
        'respiration_rate (BrPM)': respiration_rates
    })

    return data

In [40]:
all_infants_data = pd.DataFrame()

for i in range(1,11):
    infant_data = process_infant_data(i)
    infant_data['infant_num'] = i
    all_infants_data = pd.concat([all_infants_data, infant_data], ignore_index=True)

In [41]:
all_infants_data.to_csv('all_infants_vital_signs.csv', index=False)